# [모의 캐글-의료] 흉부 CT 코로나 감염 여부 분류
- 이미지 binary 분류 과제
- 담당: 이녕민M

## Import Libraries

In [ ]:
# !apt-get update && apt-get install -y python3-opencv

In [ ]:
# !pip install sklearn

In [5]:
import os, torch, copy, cv2, sys, random
# from datetime import datetime, timezone, timedelta
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

## Set Arguments & hyperparameters

In [6]:
# 시드(seed) 설정

RANDOM_SEED = 2022

torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

In [7]:
# parameters

### 데이터 디렉토리 설정 ###
DATA_DIR= 'data'
NUM_CLS = 2

EPOCHS = 30
BATCH_SIZE = 32
LEARNING_RATE = 0.0005
EARLY_STOPPING_PATIENCE = 10
INPUT_SHAPE = 128

os.environ["CUDA_VISIBLE_DEVICES"]="0"
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Dataloader

#### Train & Validation Set loader

In [8]:
class CustomDataset(Dataset):
    def __init__(self, data_dir, mode, input_shape):
        self.data_dir = data_dir
        self.mode = mode
        self.input_shape = input_shape
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Dataset split
        if self.mode == 'train':
            self.db = self.db[:int(len(self.db) * 0.9)]
        elif self.mode == 'val':
            self.db = self.db[int(len(self.db) * 0.9):]
            self.db.reset_index(inplace=True)
        else:
            print(f'!!! Invalid split {self.mode}... !!!')
            
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def data_loader(self):
        print('Loading ' + self.mode + ' dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        # (COVID : 1, No : 0)
        db = pd.read_csv(os.path.join(self.data_dir, 'train.csv'))
        
        return db

    def __len__(self):
        return len(self.db)

    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])

        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'train',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))

        return trans_image, data['COVID']


## Model

In [9]:
import torch.nn.functional as F

class custom_CNN(nn.Module):
    def __init__(self, num_classes):
        super(custom_CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=25, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=25*29*29, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=num_classes)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) # (32, 3, 128, 128) -> (32, 8, 62, 62)
        x = self.pool(F.relu(self.conv2(x))) # (32, 8, 62, 62) -> (32, 25, 29, 29)
        
        x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        output = self.softmax(x)
        
        return output

## Utils
### EarlyStopper

In [10]:
class LossEarlyStopper():
    """Early stopper
    
    Attributes:
        patience (int): loss가 줄어들지 않아도 학습할 epoch 수
        patience_counter (int): loss 가 줄어들지 않을 때 마다 1씩 증가, 감소 시 0으로 리셋
        min_loss (float): 최소 loss
        stop (bool): True 일 때 학습 중단

    """

    def __init__(self, patience: int)-> None:
        self.patience = patience

        self.patience_counter = 0
        self.min_loss = np.Inf
        self.stop = False
        self.save_model = False

    def check_early_stopping(self, loss: float)-> None:
        """Early stopping 여부 판단"""  

        if self.min_loss == np.Inf:
            self.min_loss = loss
            return None

        elif loss > self.min_loss:
            self.patience_counter += 1
            msg = f"Early stopping counter {self.patience_counter}/{self.patience}"

            if self.patience_counter == self.patience:
                self.stop = True
                
        elif loss <= self.min_loss:
            self.patience_counter = 0
            self.save_model = True
            msg = f"Validation loss decreased {self.min_loss} -> {loss}"
            self.min_loss = loss
        
        print(msg)

### Trainer

In [11]:
class Trainer():
    """ epoch에 대한 학습 및 검증 절차 정의"""
    
    def __init__(self, loss_fn, model, device, metric_fn, optimizer=None, scheduler=None):
        """ 초기화
        """
        self.loss_fn = loss_fn
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.metric_fn = metric_fn

    def train_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 학습 절차"""
        
        self.model.train()
        train_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            
            pred = self.model(img)
            
            loss = self.loss_fn(pred[:,1], label)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            self.scheduler.step()
            
            train_total_loss += loss.item()
            prob_lst.extend(pred[:, 1].cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
        self.train_mean_loss = train_total_loss / batch_index
        self.train_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Epoch {epoch_index}, Train loss: {self.train_mean_loss}, Acc: {self.train_score}, F1-Macro: {f1}'
        print(msg)

    def validate_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 검증 절차
        """
        self.model.eval()
        val_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            pred = self.model(img)
            
            loss = self.loss_fn(pred[:,1], label)
            val_total_loss += loss.item()
            prob_lst.extend(pred[:, 1].cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
        self.val_mean_loss = val_total_loss / batch_index
        self.validation_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Epoch {epoch_index}, Val loss: {self.val_mean_loss}, Acc: {self.validation_score}, F1-Macro: {f1}'
        print(msg)

### Metrics

In [12]:
from sklearn.metrics import accuracy_score, f1_score

def get_metric_fn(y_pred, y_answer):
    """ 성능을 반환하는 함수"""
    
    assert len(y_pred) == len(y_answer), 'The size of prediction and answer are not same.'
    accuracy = accuracy_score(y_answer, y_pred)
    f1 = f1_score(y_answer, y_pred, average='macro')
    return accuracy, f1

## Train
### 학습을 위한 객체 선언

#### Load Dataset & Dataloader

In [13]:
# Load dataset & dataloader
train_dataset = CustomDataset(data_dir=DATA_DIR, mode='train', input_shape=INPUT_SHAPE)
validation_dataset = CustomDataset(data_dir=DATA_DIR, mode='val', input_shape=INPUT_SHAPE)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True)
print('Train set samples:',len(train_dataset),  'Val set samples:', len(validation_dataset))

Loading train dataset..
Loading val dataset..
Train set samples: 581 Val set samples: 65


#### Load model and other utils

In [14]:
# Load Model
model = custom_CNN(NUM_CLS).to(DEVICE)

# # Save Initial Model
# torch.save(model.state_dict(), 'initial.pt')

# Set optimizer, scheduler, loss function, metric function
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler =  optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e5, max_lr=0.0001, epochs=EPOCHS, steps_per_epoch=len(train_dataloader))
loss_fn = nn.BCELoss()
metric_fn = get_metric_fn


# Set trainer
trainer = Trainer(loss_fn, model, DEVICE, metric_fn, optimizer, scheduler)

# Set earlystopper
early_stopper = LossEarlyStopper(patience=EARLY_STOPPING_PATIENCE)

In [15]:
model

custom_CNN(
  (conv1): Conv2d(3, 8, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 25, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=21025, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
  (softmax): Softmax(dim=1)
)

### epoch 단위 학습 진행

In [16]:
for epoch_index in tqdm(range(EPOCHS)):

    trainer.train_epoch(train_dataloader, epoch_index)
    trainer.validate_epoch(validation_dataloader, epoch_index)

    # early_stopping check
    early_stopper.check_early_stopping(loss=trainer.val_mean_loss)

    if early_stopper.stop:
        print('Early stopped')
        break

    if early_stopper.save_model:
        check_point = {
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict()
        }
        torch.save(check_point, 'best.pt')


  0% 0/30 [00:00<?, ?it/s]

Epoch 0, Train loss: 0.7280382547113631, Acc: 0.5335628227194492, F1-Macro: 0.3545106077687814


  3% 1/30 [00:43<21:14, 43.95s/it]

Epoch 0, Val loss: 1.028156965970993, Acc: 0.5538461538461539, F1-Macro: 0.45219412961348443
Epoch 1, Train loss: 0.7042654355367025, Acc: 0.6006884681583476, F1-Macro: 0.5844062947067239


  7% 2/30 [01:26<20:22, 43.68s/it]

...

Epoch 15, Train loss: 0.17618847721152836, Acc: 0.9518072289156626, F1-Macro: 0.951610984199505
Epoch 15, Val loss: 1.494243398308754, Acc: 0.7538461538461538, F1-Macro: 0.7537878787878789
Early stopping counter 9/10


 53% 16/30 [11:26<09:58, 42.78s/it]

Epoch 16, Train loss: 0.17099285539653566, Acc: 0.955249569707401, F1-Macro: 0.9550470182121176


 53% 16/30 [12:09<10:38, 45.62s/it]

Epoch 16, Val loss: 0.9956555366516113, Acc: 0.8153846153846154, F1-Macro: 0.8142857142857143
Early stopping counter 10/10
Early stopped


## Inference
### 모델 로드

In [17]:
TRAINED_MODEL_PATH = 'best.pt'

### Load dataset

In [18]:
class TestDataset(Dataset):
    def __init__(self, data_dir, input_shape):
        self.data_dir = data_dir
        self.input_shape = input_shape
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def data_loader(self):
        print('Loading test dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        db = pd.read_csv(os.path.join(self.data_dir, 'sample_submission.csv'))
        return db
    
    def __len__(self):
        return len(self.db)
    
    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])
        
        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'test',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))

        return trans_image, data['file_name']

In [19]:
# Load dataset & dataloader
test_dataset = TestDataset(data_dir=DATA_DIR, input_shape=INPUT_SHAPE)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Loading test dataset..


### 추론 진행

In [20]:
model.load_state_dict(torch.load(TRAINED_MODEL_PATH)['model'])

# Prediction
file_lst = []
pred_lst = []
prob_lst = []
model.eval()
with torch.no_grad():
    for batch_index, (img, file_num) in tqdm(enumerate(test_dataloader)):
        img = img.to(DEVICE)
        pred = model(img)
        print(pred)
        file_lst.extend(list(file_num))
        pred_lst.extend(pred.argmax(dim=1).tolist())
        prob_lst.extend(pred[:, 1].tolist())

1it [00:01,  2.00s/it]

tensor([[2.6712e-01, 7.3288e-01],
        [9.6185e-01, 3.8145e-02],
        [9.2239e-01, 7.7606e-02],
        [1.5191e-01, 8.4809e-01],
        [9.1649e-01, 8.3513e-02],
        [8.1864e-02, 9.1814e-01],
        [6.0600e-01, 3.9400e-01],
        [1.4044e-01, 8.5956e-01],
        [9.9181e-01, 8.1920e-03],
        [3.4842e-04, 9.9965e-01],
        [7.0028e-01, 2.9972e-01],
        [4.7684e-01, 5.2316e-01],
        [5.0852e-01, 4.9148e-01],
        [9.9725e-01, 2.7487e-03],
        [9.0748e-01, 9.2518e-02],
        [8.7741e-01, 1.2259e-01],
        [2.2108e-01, 7.7892e-01],
        [9.9194e-01, 8.0642e-03],
        [8.7278e-01, 1.2722e-01],
        [9.3216e-01, 6.7844e-02],
        [9.5347e-01, 4.6531e-02],
        [9.2643e-01, 7.3568e-02],
        [7.3147e-01, 2.6853e-01],
        [8.2783e-02, 9.1722e-01],
        [9.8721e-01, 1.2787e-02],
        [9.4552e-01, 5.4484e-02],
        [9.8295e-01, 1.7054e-02],
        [8.3151e-01, 1.6849e-01],
        [5.7073e-02, 9.4293e-01],
        [3.172

2it [00:04,  2.06s/it]

tensor([[4.7847e-01, 5.2153e-01],
        [4.6223e-01, 5.3777e-01],
        [9.4726e-01, 5.2737e-02],
        [4.6237e-01, 5.3763e-01],
        [9.1524e-01, 8.4759e-02],
        [4.2799e-02, 9.5720e-01],
        [4.6318e-01, 5.3682e-01],
        [4.2497e-01, 5.7503e-01],
        [6.4304e-02, 9.3570e-01],
        [3.6172e-01, 6.3828e-01],
        [9.4055e-01, 5.9446e-02],
        [1.2780e-01, 8.7220e-01],
        [3.4895e-01, 6.5105e-01],
        [8.5342e-02, 9.1466e-01],
        [2.4901e-01, 7.5099e-01],
        [9.9955e-01, 4.4822e-04],
        [7.9616e-01, 2.0384e-01],
        [6.9790e-01, 3.0210e-01],
        [3.0908e-01, 6.9092e-01],
        [3.8789e-02, 9.6121e-01],
        [6.3969e-01, 3.6031e-01],
        [6.3727e-01, 3.6273e-01],
        [9.5769e-02, 9.0423e-01],
        [6.9410e-01, 3.0590e-01],
        [3.3591e-01, 6.6409e-01],
        [9.5250e-02, 9.0475e-01],
        [9.9409e-01, 5.9149e-03],
        [2.3595e-01, 7.6405e-01],
        [1.5678e-01, 8.4322e-01],
        [9.603

3it [00:06,  2.07s/it]

tensor([[0.0548, 0.9452],
        [0.0046, 0.9954],
        [0.4362, 0.5638],
        [0.9257, 0.0743],
        [0.0701, 0.9299],
        [0.9468, 0.0532],
        [0.8442, 0.1558],
        [0.9594, 0.0406],
        [0.9904, 0.0096],
        [0.9602, 0.0398],
        [0.0179, 0.9821],
        [0.9976, 0.0024],
        [0.9835, 0.0165],
        [0.9146, 0.0854],
        [0.5116, 0.4884],
        [0.3096, 0.6904],
        [0.1027, 0.8973],
        [0.8283, 0.1717],
        [0.8478, 0.1522],
        [0.2571, 0.7429],
        [0.2345, 0.7655],
        [0.6485, 0.3515],
        [0.4463, 0.5537],
        [0.9181, 0.0819],
        [0.6346, 0.3654],
        [0.3511, 0.6489],
        [0.7526, 0.2474],
        [0.9055, 0.0945],
        [0.7745, 0.2255],
        [0.0442, 0.9558],
        [0.9880, 0.0120],
        [0.5604, 0.4396]], device='cuda:0')


4it [00:06,  1.65s/it]

tensor([[0.4962, 0.5038],
        [0.4168, 0.5832],
        [0.4486, 0.5514],
        [0.3541, 0.6459]], device='cuda:0')


### 결과 저장

In [23]:
df = pd.DataFrame({'file_name':file_lst, 'COVID':pred_lst})
# df.sort_values(by=['file_name'], inplace=True)
df.to_csv('prediction.csv', index=False)